### 在Python中,一切皆对象

#### 函数式编程

- 把一个函数当作是另外一个函数的返回结果

- 函数可以当作参数

#### 闭包
在一个外函数中定义了一个内函数，内函数里运用了外函数的临时变量，并且外函数的返回值是内函数的引用。这样就构成了一个闭包。

一般情况下，在我们认知当中，如果一个函数结束，函数的内部所有东西都会释放掉，还给内存，局部变量都会消失。

但是闭包是一种特殊情况，如果外函数在结束的时候发现有自己的临时变量将来会在内部函数中用到，就把这个临时变量绑定给了内部函数，然后自己再结束。

具体可以看:https://www.cnblogs.com/s-1314-521/p/9763376.html

In [4]:
def outer():
    x = 10  # 外部函数的临时变量(也称环境变量)
    def inner(y):
        print(x + y)  # 内部函数调用了环境变量
    return inner  # 返回内部函数的引用

# f存了外函数的返回值,也就是inner函数的引用
f = outer()  

# 这里相当于执行inner函数
f(10)

20


闭包中内函数修改外函数局部变量：

　　在闭包内函数中，我们可以随意使用外函数绑定来的临时变量，但是如果我们想修改外函数临时变量数值的时候发现出问题了！

　　在基本的python语法当中，一个函数可以随意读取全局数据，但是要修改全局数据的时候有两种方法:
    
     1.global 声明全局变量 
    
     2.全局变量是可变类型数据的时候可以修改

　　在闭包内函数也是类似的情况。在内函数中想修改闭包变量（外函数绑定给内函数的局部变量）的时候：

- 在python3中，可以用nonlocal 关键字声明 一个变量， 表示这个变量不是局部变量空间的变量，需要向上一层变量空间找这个变量。

- 在python2中，没有nonlocal这个关键字，我们可以把闭包变量改成可变类型数据进行修改，比如列表。

In [1]:
# 先看下在内部函数中想修改外部变量值x的错误实例
def outer():
    x = 10 
    def inner(y):
        x += 1  # 修改x
        print(x + y)
    return inner

f = outer()  
f(10)  # 报错:local variable 'x' referenced before assignment

UnboundLocalError: local variable 'x' referenced before assignment

In [2]:
#  方法1 修改闭包变量的实例
def outer():
    x = 10
    def inner(y):
        # 内函数中想修改环境变量
        nonlocal x  # nonlocal关键字声明
        x += 1
        print("内部函数修改环境变量后:", x + y)
    return inner
f2 = outer()
f2(10)  # 成功修改了外部函数的环境变量
print(f2.__closure__)

内部函数修改环境变量后: 21
(<cell at 0x000001FA585554F8: int object at 0x00007FF964777250>,)


In [3]:

#  方法2 把闭包变量修改成可变数据类型 比如列表
def outer():
    x = 10
    z = [x]
    def inner(y):
        # 内函数中想修改环境变量
        z[0] += 1  # 把x"包装"成可变数据类型
        print("内部函数修改环境变量后:", z[0] + y)
    return inner
f2 = outer()
f2(10)  # 成功修改了外部函数的环境变量
print(f2.__closure__)

内部函数修改环境变量后: 21
(<cell at 0x000001FA585704C8: list object at 0x000001FA5856BF08>,)


### 还有一点需要注意：
使用闭包的过程中，一旦外函数被调用一次返回了内函数的引用，虽然每次调用内函数，是开启一个函数执行过后消亡，但是闭包变量实际上只有一份，每次开启内函数都在使用同一份闭包变量

In [6]:
def outer(x):
    def inner(y):
        nonlocal x
        x += y
        return x
    return inner

f3 = outer(1)
print(f3(10))

# 再次调用
print(f3(10))  # 21
# 由此可见，每次调用inner的时候，使用的闭包变量x实际上是同一个。


11
21
